In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [120]:
# Loading data
# Source: https://datamarket.com/data/set/22n4/monthly-car-sales-in-quebec-1960-1968#!ds=22n4&display=line
data = pd.read_csv('monthly-car-sales-in-quebec-1960.csv', parse_dates=['Month'])

In [121]:
data.head()

,Month,Monthly car sales in Quebec 1960-1968
0,1960-01-01,6550
1,1960-02-01,8728
2,1960-03-01,12026
3,1960-04-01,14395
4,1960-05-01,14587


In [122]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 2 columns):
Month                                    108 non-null datetime64[ns]
Monthly car sales in Quebec 1960-1968    108 non-null int64
dtypes: datetime64[ns](1), int64(1)
memory usage: 1.8 KB


In [123]:
# Forecasting parameters
n_training_months = 36
n_prediction_months = 12
n_total_months = n_training_months + n_prediction_months
print(n_total_months)

48


In [124]:
# Cutting some months of data
data = data.iloc[0:n_total_months]

In [125]:
y_train = data.iloc[0:n_training_months, 1]
y_test = data.iloc[n_training_months:, 1]
print("Training months\n")
print(y_train)
print("\n")
print("Prediction months\n")
print(y_test)

Training months

0      6550
1      8728
2     12026
3     14395
4     14587
5     13791
6      9498
7      8251
8      7049
9      9545
10     9364
11     8456
12     7237
13     9374
14    11837
15    13784
16    15926
17    13821
18    11143
19     7975
20     7610
21    10015
22    12759
23     8816
24    10677
25    10947
26    15200
27    17010
28    20900
29    16205
30    12143
31     8997
32     5568
33    11474
34    12256
35    10583
Name: Monthly car sales in Quebec 1960-1968, dtype: int64


Prediction months

36    10862
37    10965
38    14405
39    20379
40    20128
41    17816
42    12268
43     8642
44     7962
45    13932
46    15936
47    12628
Name: Monthly car sales in Quebec 1960-1968, dtype: int64


In [126]:
print(y_train.shape)
print(y_test.shape)

(36,)
(12,)


In [127]:
import numpy as np

def _load_data(data, n_prev = 100, n_next=10):  
    """
    data should be pd.DataFrame()
    """

    docX, docY = [], []
    for i in range(len(data)-n_prev-n_next):
        tempX = []
        for j in range(i, i+n_prev):
            tempX.append(data.iloc[j:j+n_prev].as_matrix())
            #tempX = data.iloc[j:j+n_prev].as_matrix()
        docX.append(tempX)
        docY.append(data.iloc[i+n_prev:(i+n_prev+n_next)].as_matrix())
    alsX = np.array(docX)
    alsY = np.array(docY)

    return alsX, alsY

In [128]:
y_train.as_matrix()

array([ 6550,  8728, 12026, 14395, 14587, 13791,  9498,  8251,  7049,
        9545,  9364,  8456,  7237,  9374, 11837, 13784, 15926, 13821,
       11143,  7975,  7610, 10015, 12759,  8816, 10677, 10947, 15200,
       17010, 20900, 16205, 12143,  8997,  5568, 11474, 12256, 10583])

In [129]:
X, y = _load_data(y_train, n_prev=6, n_next=6)
print(X[0])
print("------")
print(y[0])

[[ 6550  8728 12026 14395 14587 13791]
 [ 8728 12026 14395 14587 13791  9498]
 [12026 14395 14587 13791  9498  8251]
 [14395 14587 13791  9498  8251  7049]
 [14587 13791  9498  8251  7049  9545]
 [13791  9498  8251  7049  9545  9364]]
------
[9498 8251 7049 9545 9364 8456]


In [130]:
print(X.shape)
print(y.shape)

(24, 6, 6)
(24, 6)


In [131]:
timesteps = X.shape[1]
print(timesteps)

6


In [132]:
input_dim = X.shape[2]
print(input_dim)

6


In [133]:
output_dim = y[0].shape[0]
print(output_dim)

6


In [134]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM

In [199]:
model = Sequential()
model.add(LSTM(1500, input_dim=input_dim, input_length=timesteps))
model.add(Dense(output_dim, activation='relu'))
#model.add(Dense(output_dim, activation='linear'))
#model.add(Dense(output_dim, activation='tanh'))
#model.add(Dense(output_dim, activation='softmax'))
#model.add(Dense(output_dim, activation='sigmoid'))
model.compile(loss="mape", optimizer="rmsprop")

In [196]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
lstm_18 (LSTM)                     (None, 1500)        9042000     lstm_input_18[0][0]              
____________________________________________________________________________________________________
dense_18 (Dense)                   (None, 6)           9006        lstm_18[0][0]                    
Total params: 9051006
____________________________________________________________________________________________________


In [197]:
model.fit(X, y, nb_epoch=1000)

Epoch 1/1000
24/24 [==============================] - 1s - loss: 99.9973
Epoch 2/1000
24/24 [==============================] - 1s - loss: 99.9941
Epoch 3/1000
24/24 [==============================] - 1s - loss: 99.9918
Epoch 4/1000
24/24 [==============================] - 1s - loss: 99.9892
Epoch 5/1000
24/24 [==============================] - 1s - loss: 99.9865
Epoch 6/1000
24/24 [==============================] - 1s - loss: 99.9839
Epoch 7/1000
24/24 [==============================] - 1s - loss: 99.9812
Epoch 8/1000
24/24 [==============================] - 1s - loss: 99.9787
Epoch 9/1000
24/24 [==============================] - 1s - loss: 99.9762
Epoch 10/1000
24/24 [==============================] - 1s - loss: 99.9738
Epoch 11/1000
24/24 [==============================] - 1s - loss: 99.9714
Epoch 12/1000
24/24 [==============================] - 1s - loss: 99.9691
Epoch 13/1000
24/24 [==============================] - 1s - loss: 99.9667
Epoch 14/1000
24/24 [==========================

In [198]:
model.predict(X)[0]

array([ 221.60195923,  223.63893127,  225.72027588,  225.31416321,
        221.71409607,  220.40003967])

In [170]:
y[0]

array([9498, 8251, 7049, 9545, 9364, 8456])